In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import Timestamp
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

In [ ]:
# this csv only goes back to 2014
# df = pd.read_csv('../pricehistory/price_full.csv')
# btcmarketcap = df[['Start','Market Cap']]
# btcmarketcap.set_index('Start', inplace=True)
# btcmarketcap = btcmarketcap.iloc[::-1]
# btcmarketcap = btcmarketcap[btcmarketcap['Market Cap'] != 0]
# btcmarketcap["day"] = btcmarketcap.index
# btcmarketcap.reset_index(drop=True, inplace=True)
# btcmarketcap

In [ ]:
df = pd.read_csv('bitcoin@public.csv',delimiter=';')
btcmarketcap = df[["CapMrktCurUSD","Date"]].dropna()
btcmarketcap.rename(columns={"CapMrktCurUSD":"Market Cap","Date":"day"},inplace=True)
btcmarketcap["day"] = pd.to_datetime(btcmarketcap["day"])
btcmarketcap.index = btcmarketcap["day"]
btcmarketcap = btcmarketcap[["Market Cap"]]

# create a january first for each year and interpolate the market cap
for year in range(2014,2022):
    btcmarketcap.loc[pd.to_datetime(f"{year}-01-01")] = np.nan
btcmarketcap.sort_index(inplace=True)
btcmarketcap.interpolate(inplace=True)

btcmarketcap["day"] = btcmarketcap.index
# looks like
# 4730	6.864460e+11	2021-06-01
# 4731	5.947511e+11	2021-07-15
btcmarketcap

In [ ]:
# plot btcmarketcap with log10 y axis, x axis is year and at a 30 degree angle
fig, ax = plt.subplots()
btcmarketcap.plot(ax=ax)
ax.set_yscale('log')
# since the days are not evenly spaced, we can't use np.arange
# plt.xticks([btcmarketcap.index[0],btcmarketcap.index[-1]], [str(btcmarketcap["day"].iloc[0])[:4],str(btcmarketcap["day"].iloc[-1])[:4]])
plt.xticks(rotation=30)
plt.show()

In [ ]:
# from https://data.un.org/
gdp = pd.read_csv('gdp.csv', encoding='ISO-8859-1') # Country,Year,TypeOfValue,Value
gdp = gdp[gdp['TypeOfValue'] == 'GDP in current prices (millions of US dollars)']
gdp = gdp[["Country","Year","Value"]]
gdp = gdp.reset_index(drop=True)
# ignore all rows that come before the first row that has "Afghanistan" in the country column
gdp = gdp.iloc[gdp[gdp['Country'] == 'Afghanistan'].index[0]:]
gdp = gdp.reset_index(drop=True)
gdp["Value"] = gdp["Value"].str.replace(",","").astype(float)*1e6
gdp # now looks like: 0	Afghanistan	1995 2.757000e+09

In [ ]:
# Step 1: Take the natural logarithm of every value in the 'Value' and 'Market Cap' columns
gdp['ln_Value'] = np.log(gdp['Value'])
btcmarketcap['ln_Market_Cap'] = np.log(btcmarketcap['Market Cap'])

first_year = 2011

# Step 2: Interpolate missing GDP values for each country from first_year to 2021
# First, pivot the gdp dataframe for easier manipulation
gdp_pivot = gdp.pivot(index='Country', columns='Year', values='ln_Value')

# Interpolate linearly across the years for each country
gdp_interpolated = gdp_pivot.reindex(columns=range(1995, 2022)).interpolate(axis=1)

# Filter out the years we are interested in (first_year-2021)
gdp_interpolated = gdp_interpolated.loc[:, first_year:2021]

# Step 3: Extract Bitcoin market cap for each January 1st from first_year to 2021
btcmarketcap['Date'] = pd.to_datetime(btcmarketcap['day'])
btc_jan_1st = btcmarketcap[btcmarketcap['Date'].dt.month == 1]
btc_jan_1st = btc_jan_1st[btc_jan_1st['Date'].dt.day == 1]
btc_jan_1st = btc_jan_1st.set_index(btc_jan_1st['Date'].dt.year)
btc_jan_1st = btc_jan_1st.loc[first_year:2021, 'ln_Market_Cap']

# Step 4: Rank Bitcoin market cap among the countries' GDPs and plot the rankings
rankings = pd.DataFrame(index=range(first_year, 2022))
for year in range(first_year, 2022):
    year_gdp = gdp_interpolated[year].dropna()
    btc_cap = btc_jan_1st.loc[year]
    rank = (year_gdp < btc_cap).sum() + 1  # Ranking position; +1 because rank starts from 1
    rankings.loc[year, 'Rank'] = rank

# Step 5, take the amount of countries and substrack the rankings
countries_count = gdp_interpolated.shape[0]
rankings['Rank'] = countries_count - rankings['Rank'] + 1

# Plotting the rankings
plt.figure(figsize=(10, 6))
plt.plot(rankings.index, rankings['Rank'], marker='o')
plt.title('Bitcoin Market Cap Ranking Among Countries by GDP')
plt.xlabel('Year')
plt.ylabel('Rank (Top x)')
plt.grid(True)
# invert y axis
plt.gca().invert_yaxis()
plt.show()

In [ ]:
countries_count

In [ ]:
np.array(gdp_interpolated.index)